In [1]:
import pandas as pd

In [ ]:
test = pd.read_csv('/Users/t2023-m0149/Documents/spartacodingclub/Projects/4. insurance/test.csv')

In [ ]:
sample = pd.read_csv('/Users/t2023-m0149/Documents/spartacodingclub/Projects/4. insurance/sample.csv')

In [ ]:
train = pd.read_csv('/Users/t2023-m0149/Documents/spartacodingclub/Projects/4. insurance/train.csv')

In [15]:
print("전체 행 개수 :", len(train))
print(train.isna().sum())     

전체 행 개수 : 1200000
id                           0
Age                      18705
Gender                       0
Annual Income            44949
Marital Status           18529
Number of Dependents    109672
Education Level              0
Occupation              358075
Health Score             74076
Location                     0
Policy Type                  0
Previous Claims         364029
Vehicle Age                  6
Credit Score            137882
Insurance Duration           1
Policy Start Date            0
Customer Feedback        77824
Smoking Status               0
Exercise Frequency           0
Property Type                0
Premium Amount               0
dtype: int64


## 데이터 전처리_재시도

In [16]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

# 데이터 불러오기
df = pd.read_csv("train.csv")  # 파일 경로에 맞게 수정

# 1. 로그 변환
df['Annual Income'] = np.log1p(df['Annual Income'])

# 2. 결측치 처리
mean_fill = ['Health Score', 'Credit Score', 'Age', 'Vehicle Age']
median_fill = ['Previous Claims']

for col in mean_fill:
    df[col] = df[col].fillna(df[col].mean())

for col in median_fill:
    df[col] = df[col].fillna(df[col].median())

# 3. Label Encoding (범주형)
label_cols = [
    'Marital Status', 'Education Level', 'Policy Type', 'Gender',
    'Smoking Status', 'Number of Dependents', 'Property Type',
    'Exercise Frequency', 'Location'
]

for col in label_cols:
    df[col] = df[col].astype(str)
    df[col] = LabelEncoder().fit_transform(df[col])

# 4. 고유값 제거
df.drop(columns=['id'], inplace=True)

# 5. 날짜형 처리
df['Policy Start Date'] = pd.to_datetime(df['Policy Start Date'], errors='coerce')
df['Policy_Year'] = df['Policy Start Date'].dt.year
df['Policy_Month'] = df['Policy Start Date'].dt.month
df.drop(columns=['Policy Start Date'], inplace=True)

# 6. 다중 클래스 범주형 -> 빈도 인코딩
for col in ['Occupation', 'Customer Feedback']:
    freq_map = df[col].value_counts().to_dict()
    df[col] = df[col].map(freq_map)

# 전처리 완료
print(df.info())



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200000 entries, 0 to 1199999
Data columns (total 21 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   Age                   1200000 non-null  float64
 1   Gender                1200000 non-null  int64  
 2   Annual Income         1155051 non-null  float64
 3   Marital Status        1200000 non-null  int64  
 4   Number of Dependents  1200000 non-null  int64  
 5   Education Level       1200000 non-null  int64  
 6   Occupation            841925 non-null   float64
 7   Health Score          1200000 non-null  float64
 8   Location              1200000 non-null  int64  
 9   Policy Type           1200000 non-null  int64  
 10  Previous Claims       1200000 non-null  float64
 11  Vehicle Age           1200000 non-null  float64
 12  Credit Score          1200000 non-null  float64
 13  Insurance Duration    1199999 non-null  float64
 14  Customer Feedback     1122176 non-

In [17]:
import pandas as pd


# 1. Annual Income 결측치 제거
df = df.dropna(subset=['Annual Income'])


# 2. Occupation 결측치를 'Unknown'으로 대체
df['Occupation'] = df['Occupation'].fillna('Unknown')


# 3. Customer Feedback 결측치를 'Unknown'으로 대체
df['Customer Feedback'] = df['Customer Feedback'].fillna('Unknown')


#4.
df.drop('Insurance Duration', axis=1, inplace=True)  # 보험 청구 여부 제거
# 결과 확인
print(df.isnull().sum())


# 저장
df.to_csv("cleaned_train.csv", index=False)




Age                     0
Gender                  0
Annual Income           0
Marital Status          0
Number of Dependents    0
Education Level         0
Occupation              0
Health Score            0
Location                0
Policy Type             0
Previous Claims         0
Vehicle Age             0
Credit Score            0
Customer Feedback       0
Smoking Status          0
Exercise Frequency      0
Property Type           0
Premium Amount          0
Policy_Year             0
Policy_Month            0
dtype: int64


In [18]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# 데이터 불러오기
df = pd.read_csv("train.csv")

# 1. Annual Income 로그 변환
df['Annual Income'] = np.log1p(df['Annual Income'])

# 2. 결측치 처리: 평균, 중앙값 대체
mean_fill = ['Health Score', 'Credit Score', 'Age', 'Vehicle Age']
median_fill = ['Previous Claims']

for col in mean_fill:
    df[col] = df[col].fillna(df[col].mean())
for col in median_fill:
    df[col] = df[col].fillna(df[col].median())

# 3. 결측치 행 제거 (Annual Income)
df = df.dropna(subset=['Annual Income'])

# 4. 범주형 변수 라벨인코딩
label_cols = [
    'Marital Status', 'Education Level', 'Policy Type', 'Gender',
    'Smoking Status', 'Number of Dependents', 'Property Type',
    'Exercise Frequency', 'Location'
]
for col in label_cols:
    df[col] = df[col].astype(str)
    df[col] = LabelEncoder().fit_transform(df[col])

# 5. 다중 클래스 빈도 인코딩
for col in ['Occupation', 'Customer Feedback']:
    freq_map = df[col].value_counts().to_dict()
    df[col] = df[col].map(freq_map)

# 6. 불필요한 열 제거
df.drop(columns=['id', 'Insurance Duration', 'Policy Start Date'], inplace=True)

# 7. 특성과 타겟 분리
X = df.drop('Premium Amount', axis=1)
y = df['Premium Amount']

# 8. 데이터 분할
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 9. 랜덤 포레스트 회귀 모델 학습
model = RandomForestRegressor(
    n_estimators=200,
    max_depth=15,
    random_state=42,
    n_jobs=-1
)
model.fit(X_train, y_train)

# 10. 검증 및 평가
y_pred = model.predict(X_val)
rmse = np.sqrt(mean_squared_error(y_val, y_pred))
print(f"Validation RMSE: {rmse}")


Validation RMSE: 838.9844690163854
